In [1]:
import pandas as pd
from preprocess_data import *
import argparse 
from preprocess_data import *
import os
from string import punctuation
import numpy as np
import pandas as pd
import re 
from collections import Counter

## Read in full dataset, drop some rows to make it smaller

In [2]:
data = read_from_uris('/Users/myradeng/Documents/CS230/cs230-final/all-the-news.csv')

/Users/myradeng/Documents/CS230/cs230-final/preprocess_data.py:29: DtypeWarning: Columns (1,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv(uri1)


In [4]:
right = ['Fox News', ]
left = ['Buzzfeed News', 'CNN', 'New Yorker', 'New Republic', 'The New York Times', 'Vox', 'Vice News', 'Washington Post']
moderate = ['Axios', 'Business Insider', 'The Hill', 'Politico', 'CNBC', 'Economist', 'Politico', 'Reuters', 'The Hill']

In [7]:
right_data = data[data['publication'] == 'Fox News']
left_data = data[data['publication'].isin(left)]
moderate_data = data[data['publication'].isin(moderate)]

In [9]:
balanced_article_count = min(len(right_data), len(left_data), len(moderate_data))

In [10]:
balanced_article_count

20144

In [11]:
def modify_length(data, new_length):
    remove_n = len(data) - new_length
    drop_indices = np.random.choice(data.index, remove_n, replace=False)
    data_cropped = data.drop(drop_indices)
    return data_cropped

In [31]:
right_data_cropped = modify_length(right_data, int(balanced_article_count / 8))

In [32]:
left_data_cropped = modify_length(left_data, int(balanced_article_count / 8))

In [50]:
moderate_data_cropped = modify_length(moderate_data, int(balanced_article_count / 5))

In [225]:
data_balanced = pd.concat([right_data_cropped, moderate_data_cropped, left_data_cropped], axis=0)

## Process data

In [226]:
data_balanced = data_balanced.rename(columns={"article": "content"})

In [229]:
data_balanced_processed = process_data(data_balanced)

1


/var/folders/zp/rj2crtv10fnb7ytz_gytl7pw0000gn/T/ipykernel_58263/3087362155.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content'] = data['content'].str.replace(' +', ' ')


2
3
4
Male pronoun count: 52066
Female pronoun count: 23770


In [230]:
data_balanced_processed = data_balanced_processed.drop(columns=['Unnamed: 0.1'])

In [231]:
print(len(data_balanced_processed[data_balanced_processed['publication'] == 'Fox News']),
      len(data_balanced_processed[data_balanced_processed['publication'].isin(left)]),
      len(data_balanced_processed[data_balanced_processed['publication'].isin(moderate)]))

95132 116520 110431


### Drop some malformed rows

In [232]:
indices = []
count = 0
for row_index, row in data_balanced_processed.iterrows():
    if re.search(r"\\r", str(row)):
        print(row)               #Return type is pandas.Series
        indices.append(row_index)
        count +=1

date                                          2018-05-15 00:00:00
year                                                         2018
month                                                         5.0
day                                                            15
author                                                Brian Flood
title           ABC set to dial back political humor from pro-...
content         (ABC/Greg Gayne)\rSARA GILBERT, ROSEANNE BARR\...
url             https://www.foxnews.com/entertainment/abc-set-...
section                                                     Media
publication                                              Fox News
label                                                           0
content_list    [(ABC/Greg, Gayne), SARA, GILBERT,, ROSEANNE, ...
Name: 11333, dtype: object
date                                          2016-06-29 17:10:05
year                                                         2016
month                                            

In [233]:
indices

[11333,
 217845,
 223996,
 223997,
 223998,
 223999,
 224000,
 224001,
 224002,
 224003,
 224004,
 224005,
 224006,
 224007,
 224008,
 224009,
 224010,
 224011,
 224012,
 224013,
 224014,
 224015,
 224016]

In [234]:
data_balanced_processed.drop(index=indices,inplace=True)

In [235]:
write_data_to_dir(data_balanced_processed, '/Users/myradeng/Documents/CS230/cs230-final/data-pt-2')

Created new directory at: /Users/myradeng/Documents/CS230/cs230-final/data-pt-2
Writing data to: /Users/myradeng/Documents/CS230/cs230-final/data-pt-2


### Sanity checks

In [240]:
from datasets import load_dataset
dataset = load_dataset("myradeng/cs-230-news-v2")

Using custom data configuration myradeng--cs-230-news-v2-8c9da6f3f3e55055


Extracting data files: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 249.09it/s]


Dataset csv downloaded and prepared to /Users/myradeng/.cache/huggingface/datasets/myradeng___csv/myradeng--cs-230-news-v2-8c9da6f3f3e55055/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 170.64it/s]


In [242]:
test = pd.read_csv('/Users/myradeng/Documents/CS230/cs230-final/data-pt-2/test.csv')

In [243]:
right_data = test[test['publication'] == 'Fox News']
left_data = test[test['publication'].isin(left)]
moderate_data = test[test['publication'].isin(moderate)]

In [241]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'date', 'year', 'month', 'day', 'author', 'title', 'content', 'url', 'section', 'publication', 'label', 'content_list'],
        num_rows: 257648
    })
    test: Dataset({
        features: ['Unnamed: 0', 'date', 'year', 'month', 'day', 'author', 'title', 'content', 'url', 'section', 'publication', 'label', 'content_list'],
        num_rows: 32206
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'date', 'year', 'month', 'day', 'author', 'title', 'content', 'url', 'section', 'publication', 'label', 'content_list'],
        num_rows: 32206
    })
})

## Functions

In [106]:
import contractions

In [97]:
def write_data_to_dir(data, output_dir):
    # Shuffle and split data intro train, dev, test 
    data = shuffle_data(data)
    split_1 = int(0.8 * len(data))
    split_2 = int(0.9 * len(data))
    train_data = data[:split_1]
    dev_data = data[split_1:split_2]
    test_data = data[split_2:]

    # Write data out to three separate files under output dir 
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print('Created new directory at: ' + output_dir)
    print('Writing data to: ' + output_dir)
    train_data.to_csv(output_dir + '/train.csv')
    dev_data.to_csv(output_dir + '/dev.csv')
    test_data.to_csv(output_dir + '/test.csv')

In [228]:
def process_data(data):
    #output_dir = '/Users/myradeng/Documents/CS230/cs-230-final/' + data_type + '-data'
    filter_gender = False
    proportion_gender = False
    ratio = 1.3
    
    print("1")
    data = data.drop(columns=['Unnamed: 0'])
    data = data[data['content'].notna()]
    data['content'] = data['content'].str.replace(' +', ' ')
    data['label'] = 0
    

    print("2")
    # Split data entries into separate sentences and remove empty strings
    data = split_df_list(data, 'content', '.')
    data = data[data['content'].str.len() > 0]
    print("3")
    data['content'] = np.where(data['content'].str.endswith('.'), data['content'], data['content'] + '.')
    data['content'] = data['content'].str.strip()
    data['content'] = data['content'].astype(str)
    data['content'] = data['content'].apply(lambda x: contractions.fix(x))

    data['content' + "_list"] = data['content'].str.split()
    print("4")
    # (Configurable) Filter for single instance pronoun entries
    if filter_gender:
        data['pronoun_count'] = data.apply(lambda row: sum([item in row['content_list'] for item in woman_keywords + man_keywords]), axis=1)
        data = data[data['pronoun_count'] == 1]

    # Count initial gender proportions
    female_pronoun_count, male_pronoun_count = count_gender_entries(data)
    print('Male pronoun count: ' + str(male_pronoun_count))
    print('Female pronoun count: ' + str(female_pronoun_count))

    # Ensure approx equal split between male / female gender pronouns in training data
    if proportion_gender:
        data = proportion_gender_entries(data, male_pronoun_count, female_pronoun_count, ratio)
        female_pronoun_count, male_pronoun_count = count_gender_entries(data)
        print('Male pronoun count after rebalancing: ' + str(male_pronoun_count))
        print('Female pronoun count after rebalancing: ' + str(female_pronoun_count))
    
    return data

In [227]:
woman_keywords = ['woman', 'women', 'female', 'she', 'her', 'hers']
man_keywords = ['man', 'men', 'male', 'he', 'his', 'him']
CONTENT_COLUMN_NAME = 'content'


def shuffle_data(df, seed=0):
	return df.sample(frac=1, random_state=0)

def parse_args():
	parser = argparse.ArgumentParser(description='Process input files into train, dev, test')
	parser.add_argument('-i', dest='input_uri', type=str)
	parser.add_argument('-o', dest='output_dir', type=str)
	# Whether to filter for only sentences with a gender pronoun present 
	parser.add_argument('--filter_gender', default=False, action=argparse.BooleanOptionalAction)
	# This is a hyperparameter used to tune how many rows to drop in order to achieve roughly ~50/50 pronoun balance
	parser.add_argument('-r', dest='ratio_param', type=str, default=1.35)
	return parser

def filter_gender_entries(data):
	gender_keyword_filtered_df = data[data[CONTENT_COLUMN_NAME + "_list"].apply(lambda x: any([k in x for k in woman_keywords]))]
	gender_keyword_filtered_df = gender_keyword_filtered_df[gender_keyword_filtered_df[CONTENT_COLUMN_NAME + "_list"].apply(lambda x: any([k in x for k in man_keywords]))]
	return gender_keyword_filtered_df

# TODO: Make this more systematic rather than an approximation
def proportion_gender_entries(data, male_pronoun_count, female_pronoun_count, ratio):
	women = data[data['content_list'].apply(lambda x: any([k in x for k in woman_keywords]))]
	men = data[data['content_list'].apply(lambda x: any([k in x for k in man_keywords]))]
	men_with_women = men[men['content_list'].apply(lambda x: any([k in x for k in woman_keywords]))]
	men_no_women = men[men['content_list'].apply(lambda x: any([k in x for k in woman_keywords]) == False)]

	remove_n = int((male_pronoun_count - female_pronoun_count) / ratio)
	drop_indices = np.random.choice(men_no_women.index, remove_n, replace=False)
	men_no_women_subset = men_no_women.drop(drop_indices)
	rebalanced = pd.concat([women, men_no_women_subset], axis=0)
	return rebalanced

def count_gender_entries(data):
	series = pd.Series(Counter([y for x in data[CONTENT_COLUMN_NAME + "_list"] for y in x]))
	female_pronoun_count = 0
	male_pronoun_count = 0 
	for female_pronoun in woman_keywords:
		female_pronoun_count += series[female_pronoun]
	for male_pronoun in man_keywords:
		male_pronoun_count += series[male_pronoun]

	return female_pronoun_count, male_pronoun_count